### ДЗ Урок 3.

Распарсить список вакансий с hh.ru постранично.

In [1]:
from bs4 import BeautifulSoup as bs
from pprint import pprint
import requests
from time import sleep

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
}

url = 'https://voronezh.hh.ru/vacancies/razrabotchik'

In [3]:
def parse(url_page, headers, result=[], index_page=1):
    
    response = requests.get(url_page, headers=headers)
    
    if response.status_code != 200:
      print('Parsing ended')
      return result
    else:
      print(f'Page{index_page} {response.url}')

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://voronezh.hh.ru' + link_next_page['href']
    else:
      print('Parsing ended')
      return result

    result = parse(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy(dom_vacancy):
  sleep(0.2)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'name': vacancy_name,
      'salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'link': vacancy_link,
      'source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [4]:
result = parse(url, headers)

Page1 https://voronezh.hh.ru/vacancies/razrabotchik
Page2 https://voronezh.hh.ru/vacancies/razrabotchik?page=1&hhtmFrom=vacancy_search_catalog
Page3 https://voronezh.hh.ru/vacancies/razrabotchik?page=2&hhtmFrom=vacancy_search_catalog
Page4 https://voronezh.hh.ru/vacancies/razrabotchik?page=3&hhtmFrom=vacancy_search_catalog
Page5 https://voronezh.hh.ru/vacancies/razrabotchik?page=4&hhtmFrom=vacancy_search_catalog
Page6 https://voronezh.hh.ru/vacancies/razrabotchik?page=5&hhtmFrom=vacancy_search_catalog
Page7 https://voronezh.hh.ru/vacancies/razrabotchik?page=6&hhtmFrom=vacancy_search_catalog
Page8 https://voronezh.hh.ru/vacancies/razrabotchik?page=7&hhtmFrom=vacancy_search_catalog
Page9 https://voronezh.hh.ru/vacancies/razrabotchik?page=8&hhtmFrom=vacancy_search_catalog
Page10 https://voronezh.hh.ru/vacancies/razrabotchik?page=9&hhtmFrom=vacancy_search_catalog
Parsing ended


In [5]:
result

[{'name': 'Младший аналитик-программист SQL',
  'salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'link': 'https://voronezh.hh.ru/vacancy/73995601?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
  'source': 'hh.ru'},
 {'name': 'Backend-разработчик / Python developer',
  'salary': '100\u202f000 – 200\u202f000 руб.',
  'min_salary': 100000,
  'max_salary': 200000,
  'currency_salary': 'руб.',
  'link': 'https://voronezh.hh.ru/vacancy/74109649?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
  'source': 'hh.ru'},
 {'name': 'Full stack разработчик',
  'salary': 'от 80\u202f000 руб.',
  'min_salary': 80000,
  'max_salary': None,
  'currency_salary': 'руб.',
  'link': 'https://voronezh.hh.ru/vacancy/73581131?query=%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA',
  'source': 'hh.ru'},
 {'name': 'Программист',
  'salary': 'от 120\u202f000 руб.',
  'min_salary': 120000,
  'max_salary'